# Supergruppe diskussion

REVISIONS||
---------||
2019-0128|CEF, initial.
2020-0295|CEF, F20 ITMAL update.

## § 2 "End-to-End Machine Learning Project" [HOML]

Genlæs kapitel (eksklusiv"Create the Workspace" og "Download the Data"), og forbered mundtlig præsentation.

Lav et kort resume af de enkelte underafsnit, ca. 5 til 20 liners tekst.

Husk at relater til "The Map":

<img src="https://itundervisning.ase.au.dk/F20_itmal/L03/Figs/ml_supervised_map.png" style="width:400px">

Kapitler (incl. underkapitler):

* Look at the Big Picture
* Get the Data (eksklusiv Create the Workspace og Download the Data),
* Discover and Visualize the Data to Gain Insights,
* Prepare the Data for Machine Learning Algorithms,
* Select and Train a Model,
* Fine-Tune Your Model,
* Launch, Monitor, and Maintain Your System,
* Try It Out!.

#### Resume: Look at the Big Picture

Kapitlet gennemgår et komplet eksempel på et Maskinlærings projekt. Der laves en model, som skal lære fra rigtige data for huspriser i Californien. Modellen skal være i stand til at forudsige medianhusprisen i et distrikt givet andre data for distriktet. Første skridt er at se på konteksten, herunder hvad modellen skal bruges til, da det påvirker hvilke algoritmer man bruger, hvordan modellen evalueres og hvordan den skal tweakes. I dette tilfælde skal modellens output processeres af et andet system for at afgøre, om distriktet er værd at investere i. Dvs. det skal indgå i en pipeline (særskilte komponenter hvis input er output fra andre komponenter, evt. pulles input fra en database). Steps:

Frame problemet: Der er tale om Supervised Learning (anvender labeled data) og univariat regression (forudsiger en enkelt værdi per distrikt). <br>Vælg performance measure: Her vælges RMSE, som er et mål for, hvor meget modellens forudsigelser afviger fra de egentlige target værdier. Alternativt kunne vælges MAE (begge mål beregner afstanden mellem vektorer, men MAE er mindre følsom overfor outliers).
<br>Check antagelser: Sørg for at bekræfte dine antagelser før det egentlige arbejde påbegyndes.

#### Resume: Get the Data (eksklusiv Create the Workspace og Download the Data)

Når data er anskaffet giver det mening at undersøge strukturen af data, herunder antal rækker, attributtyper og antal værdier, der ikke er null - hertil anvendes info() metode på datasættet. Numeriske attributter kan undersøges med describe() (giver et overblik over min, max, middelværdi, std. afvigelser og percentiler). Kategoriske attributter kan undersøges med value_counts(). Anvend eventuelt Matplotlib's hist() metode på datasættet for at tegne histogrammer for samtlige numeriske attributter. Det afslører bl.a., om data er afgrænset. Hvis det udgør et problem, kan det håndteres ved at skaffe bedre data, eller ekskludere visse værdier fra datasættet.

Herefter tilsidesættes ca. 20% af datasættet til testformål (mindre hvis datasættet er stort). Data som skal indgå i testsættet bør udvælges tilfældigt, og opdelingen skal ske konsekvent, dvs. træningssæt og testsæt skal vælges på samme måde, hver gang programmet køres (og i tilfælde af, at datasættet opdateres). Data til testsættet kan udvælges tilfældigt ud fra unikke ID'er, som for eksempel rækkenummer eller en kombination af attributter (random sampling). Det er dog vigtigt at sikre, at testsættet er repræsentativt. Særligt vigtige attributter, for eksempel median indkomst, kan opdeles i subsets/stratums, og der udvælges så et passende antal instanser fra hver stratum til testsættet (stratified sampling).

#### Resume: Discover and Visualize the Data to Gain Insights,

Nu kan data visualiseres. Bemærk der arbejdes på træningssættet eller kopier af dette. For eksempel kan distrikter afbildes i et scatterplot med x=longitude og y=latitude. Der findes korrelationskoefficienter mellem alle attributter med metoden corr(). Det er særligt interessant hvilke attributter, der har en stærk korrelation med target attributten, dvs. en koefficient tæt på 1 eller -1. Ved at plotte korrelerede værdier kan datasættet undersøges nærmere for trends, og man kan forsøge at rense datasættet for uhensigtsmæssige forhold. 
<br>Eksperimenter med attributkombinationer: Endelig kan man skabe nye attributter ud fra de eksisterende og gentage ovenstående fremgangsmåde for at se, hvor informative de er. Eksempelvis kan anvendes antal rum per husstand frem for det totale antal rum i distriktet. Det er en iterativ proces, der gerne skulle resultere i en rimelig prototype.

#### Resume: Prepare the Data for Machine Learning Algorithms

Forbereder nu data til maskinlæringsalgoritmerne. Starter med at separere predictors og labels, da de ikke nødvendigvis skal gennemgå de samme transformationer.

Data cleaning: Først ryddes op i datasættet, da algoritmerne ikke kan håndtere manglende features. Ved manglende værdier for numeriske attributter kan det håndteres på følgende måder: Fjern de tilsvarende distrikter, fjern hele attributten, eller sæt de manglende værdier (0, median, middelværdi). Med sidstnævnte tilgang anvendes f.eks. Scikit-Learn's SimpleImputer, som kan fittes til træningsdata. Den trænede imputer kan beregne medianen for hver attribut og anvendes herefter til at transformere træningssættet. Værdierne gemmes, så de senere kan anvendes på testsættet.

Håndtering af tekst- og kategoriattributter: Vil gerne konvertere disse til numeriske attributter. Kan f.eks. lave én binær attribut per kategori, og kun én af disse dummies vil være 1 for en given instans, mens de andre er 0 (one-hot encoding). Med mange mulige kategorier vil det store antal features måske påvirke performance, og da bør det overvejes helt at erstatte den kategoriske attribut med en numerisk attribut.

Custrom transformers: Det kan være nødvendigt selv at implementere en transformer, hvis man skal udføre særlige cleanup operationer eller kombinere attributter. Implementeres som en klasse med metoderne fit(), transform() og fit_transform(). Man kan med fordel tilføje TransformerMixin og BaseEstimator som basisklasser.

Feature scaling: Generelt er man nødt til at skalere attributterne. Min-max eller normalisering betyder, at man skalerer værdierne, så de ligger mellem 0 og 1. I praksis trækker man minimum fra værdierne og dividerer med max minus min. Alternativt kan der skaleres med standardisering, dvs. man trækker middelværdien fra og dividerer med standardafvigelsen. Standardisering påvirkes mindre af outliers, og værdierne er ikke bundet til et bestemt interval.

Transformation pipelines: Scikit-Learn tilbyder en Pipeline klasse, som kan hjælpe med sekvenser af transformationer. Den tager navn/estimator par i constructoren (hvor estimatorer er objekter, der kan estimere parametre ud fra et datasæt), og ved at kalde fit() på pipelinen bliver fit_transform() kaldt sekventielt samtidig med at output videregives ved hvert kald. Der eksisterer desuden en ColumnTransformer klasse, som kan bruges, hvis forskellige attributter skal gennemgå forskellige transformationer/pipelines, f.eks. numeriske og kategoriske attributter.

#### Resume: Select and Train a Model

Der vælges nu en model, f.eks. LinearRegression() eller DecisionTreeRegressor(), og denne trænes ud fra datasættet (ved at kalde fit()). Modellen kan i dette tilfælde evalueres ved at forudsige nogle værdier og sammenholde dem med de faktiske værdier (ved at beregne RMSE). En anden måde at evaluere modellerne på er med Cross-Validation. Her splitter man træningssættet i n forskellige subsets kaldet folder, dvs. man træner og evaluerer modellen n gange, hvilket resulterer i n forskellige scores og en standardafvigelse. Det kan give et billede af, om modellen underfitter eller overfitter data. Overfitting kan afhjælpes ved at simplificere modellen, begrænse den eller få fat i mere træningsdata. Målet er at afprøve en masse forskellige modeller, hvorefter man kan gå videre med de mest lovende modeller. Dette step fremgår øverst til venstre i "The Map" som hhv. "model selection" og "train".

#### Resume: Fine-Tune Your Model

Næste skridt er at finjustere modellerne, for at finde den bedst mulige kombination af hyperparametre. Dette kan gøres manuelt, men der kan med fordel anvendes Scikit-Learn's GridSearchCV til søgningen. Man specificerer hvilke kombinationer af hyperparameter værdier, der skal undersøges, og grid search vil træne modellen, hvorved man finder frem den bedste kombination af parametre. Det er ligeledes muligt at få oplyst den bedste estimator og scores. Hvis der er mange kombinationer af hyperparametre, bør man anvende RandomizedSearchCV. Den søger et givet antal tilfældige kombinationer for hvert parameter, og man kan sætte antal iterationer. Det er værd at inspicere de bedste modeller og og forsøge at mindske deres fejl, f.eks. ved at tilføje ekstra features, fjerne features og outliers eller ved at kombinere modeller (som det er tilfældet med RandomForestRegressor).

Dette step vises øverst i "The Map", som en form for feedback loop, hvor man forsøger at minimere en loss funktion. Det gentages indtil man har et tilfredsstillende resultat, hvorefter den endelige model kan evalueres på testsættet. Der anvendes predictors og labels fra testsættet og data transformeres ved at kalde transform() på den fulde pipeline (ikke fit_transform(), da testsættet ikke skal fittes). Herefter findes de predikterede værdier og de sammenholdes med testsættets labels ved at finde RMSE.

#### Resume: Launch, Monitor, and Maintain Your System

Deploy: Når løsningen er produktionsklar kan den trænede Scikit-learn model inklusiv præprocessering og pipeline gemmes med joblib. I koden anvendes modellen ved at kalde dens predict() metode, f.eks. i en webapplikation, som modtager queries med distriktsdata. Alternativt kan modellen wrappes i en webservice, som applikationen så kan sende REST kald til. En anden løsning er at deploye modellen i clouden, ved at gemme den med joblib og uploade den til en form for cloud storage. Dermed kan man få en webservice, som kan modtage distriktsdata og returnere predikterede værdier i JSON format.

Monitor: Det er vigtigt løbende at monitorere modellen, da data kan ændre sig og dens performance kan falde over tid. Afhængig af modellen kan man måske afgøre det direkte ved at inspicere/analysere resultaterne, f.eks. hvis modellen skal bruges til foreslå eller klassificere ting. Hvis performance falder kan det skyldes, at pipeline er i stykker, eller at modellen skal trænes igen med ny data. En anden måde at opdage en faldende performance på, er ved at undersøge kvaliteten af input data (f.eks. et stigende antal outliers eller manglende features). Hvis data løbende ændrer sig, bør man automatisere så meget som muligt, f.eks. ved at lave scripts for indsamling af frisk data, træning af modellen, finjustering af hyperparametre, evaluering på testsæt og deployment.

#### Resume: Try It Out!.

Forberedelsen af data udgør altså en stor del af arbejdet. Der skal desuden bygges monitorerings værktøjer, opsættes pipelines, automatiseres regelmæssig modeltræning osv.